# Data Preperation for the Project - Historical MRTS Sales Analysis

**Satish Nalla**

with the various Years of data for each Kind of business with multiple Adjustment Types

# Index

- [Abstract](#Abstract)
- [1. Initialization](#1.-Initialization)
- [2. Importing the Data](#2.-Importing-the-Data)
- [3. Exploring the Data](#3.-Exploring-the-Data)
- [4. Finalizing the Data](#4.-Finalizing-the-data-and-preparing-the-Installation-Scripts)
- [5. Conclusion](#5.-Conclusion)
- [6. References](#References)

[Back to top](#Index)
## Abstract

In order to import the MRTS Sales data which has one worksheet per year and the data is more in reporting format with various columns and sections, the Goal is to cleanse the data and get into uniform format and preparing the Installation Scripts to load it into database for further Analysis and the Project.

[Back to top](#Index)

## 1. Initialization

Importing the Python Libraries Needed for this project

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.parser import parse
import math

Data File Details

In [83]:
#input excel file name
excelfile = 'mrtssales92-present.xls'

Creation of f_isDate function to check if its a date value, this will be used in the code further to check the dates.

In [84]:
#Checking the input string is a valid date or not and returns True if input is date else false.
def f_isDate(iStr):
    try: 
        parse(iStr, fuzzy=False)
        return True

    except ValueError:
        return False

[Back to top](#Index)

## 2. Importing the Data

using pandas getting the list of worksheets in the excel file

In [85]:
#Pandas has functionality for getting the list of sheets in an excel file so that will be helpful to loop through to import data
dataFile = pd.ExcelFile(excelfile)
sheets = dataFile.sheet_names

print(sheets)

['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1994', '1993', '1992']


looping through each sheet and importing data and merging to the one master raw data variable

In [86]:
#master_raw_data going to be the final data frame of all the data
master_raw_data = pd.DataFrame()

#looping through the all sheets and importing the data for each sheet
for sheet in sheets:
    raw_data_NA = ''
    raw_data_A = ''
    #as we have 2 sections of data Not Adjustment and Adjustment Types, loading the excel file seperately for each set of rows
    raw_data_NA = pd.read_excel(excelfile, sheet_name = sheet, skiprows = 4, nrows = 66)
    raw_data_A = pd.read_excel(excelfile, sheet_name = sheet, skiprows = 71, nrows = 38)
    
    raw_data_A.columns = raw_data_NA.columns

    #Replacing the text columns to proper Date format labels.
    newCols = {}
    for col in raw_data_NA.columns:
        tempCol = col
        tempCol = tempCol.strip()
        tempCol = tempCol.replace("Unnamed: 0","NAICS_Code")
        tempCol = tempCol.replace('Unnamed: 1','Kind_of_Business')
        tempCol = tempCol.replace('.','')
        tempCol = tempCol.replace('Jan','01/01/')
        tempCol = tempCol.replace('Feb','02/01/')
        tempCol = tempCol.replace('Mar','03/01/')
        tempCol = tempCol.replace('Apr','04/01/')
        tempCol = tempCol.replace('May','05/01/')
        tempCol = tempCol.replace('Jun','06/01/')
        tempCol = tempCol.replace('Jul','07/01/')
        tempCol = tempCol.replace('Aug','08/01/')
        tempCol = tempCol.replace('Sep','09/01/')
        tempCol = tempCol.replace('Oct','10/01/')
        tempCol = tempCol.replace('Nov','11/01/')
        tempCol = tempCol.replace('Dec','12/01/')
        tempCol = tempCol.replace(' ','')
        tempCol = tempCol.replace('(p)','')
        newCols[col]= tempCol
        #print(tempCol)
        
    #print(newCols)
    
    #Assigning the new columns names to both the Dataimport pandas dataframe variables and adding an additional column Adjustment to seperateout the data when we merge both the dataframes
    raw_data_NA.rename(columns=newCols,inplace = True)
    raw_data_NA = raw_data_NA.drop(raw_data_NA[(raw_data_NA.Kind_of_Business  == 'NOT ADJUSTED')].index)
    raw_data_NA['Adjustment_Type']='NOT ADJUSTED'
    
    raw_data_A.rename(columns=newCols, inplace = True)
    raw_data_A['Adjustment_Type']='ADJUSTED'

    #Concatenating the Not Adjustment and Adjustment Dataframes
    raw_data = pd.concat([raw_data_NA,raw_data_A])
    
    #Cleansing the data by updating the date datatypes, replacing NA with NANs and other string variables in numbers and dates to NANs for better usage further.
    for cols in raw_data.columns:
        if f_isDate(cols):
            tempDataFrame = pd.DataFrame()
            tempDataFrame = raw_data.loc[:, ['NAICS_Code', 'Kind_of_Business','Adjustment_Type',cols]]
            tempDataFrame = tempDataFrame.astype({'NAICS_Code':'string'})
            tempDataFrame['Date'] = pd.to_datetime(cols)
            tempDataFrame.rename(columns = {cols : 'Value'}, inplace = True)
            tempDataFrame.replace('(NA)', np.nan, inplace = True)
            tempDataFrame.replace('(S)', np.nan, inplace = True)
            #Concatenating the prepared data for each sheet to master raw data Dataframe
            master_raw_data = pd.concat([master_raw_data,tempDataFrame])
            

#Creating Additional Columns like Year & Month for better usage
master_raw_data['Year'] =  pd.DatetimeIndex(master_raw_data['Date']).year
master_raw_data['Month'] =  pd.DatetimeIndex(master_raw_data['Date']).month

#Resetting the Index as the master_raw_data is ready
master_raw_data = master_raw_data.reset_index()

[Back to top](#Index)

## 3. Exploring the Data


Checking the master data columns and details

In [87]:
master_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36050 entries, 0 to 36049
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   index             36050 non-null  int64         
 1   NAICS_Code        31150 non-null  string        
 2   Kind_of_Business  36050 non-null  object        
 3   Adjustment_Type   36050 non-null  object        
 4   Value             35447 non-null  float64       
 5   Date              36050 non-null  datetime64[ns]
 6   Year              36050 non-null  int64         
 7   Month             36050 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2), string(1)
memory usage: 2.2+ MB


Checking the top rows from master raw data

In [88]:
master_raw_data.head()

,index,NAICS_Code,Kind_of_Business,Adjustment_Type,Value,Date,Year,Month
0,1,<NA>,"Retail and food services sales, total",NOT ADJUSTED,517119.0,2021-01-01,2021,1
1,2,<NA>,Retail sales and food services excl motor vehi...,NOT ADJUSTED,412864.0,2021-01-01,2021,1
2,3,<NA>,Retail sales and food services excl gasoline s...,NOT ADJUSTED,479905.0,2021-01-01,2021,1
3,4,<NA>,Retail sales and food services excl motor vehi...,NOT ADJUSTED,375650.0,2021-01-01,2021,1
4,5,<NA>,"Retail sales, total",NOT ADJUSTED,464362.0,2021-01-01,2021,1


Checking the bottom rows from master raw data

In [89]:
master_raw_data.tail()

,index,NAICS_Code,Kind_of_Business,Adjustment_Type,Value,Date,Year,Month
36045,33,453,Miscellaneous stores retailers,ADJUSTED,5059.0,1992-12-01,1992,12
36046,34,454,Nonstore retailers,ADJUSTED,6597.0,1992-12-01,1992,12
36047,35,4541,Electronic shopping and mail order houses,ADJUSTED,3032.0,1992-12-01,1992,12
36048,36,45431,Fuel dealers,ADJUSTED,1444.0,1992-12-01,1992,12
36049,37,722,Food services and drinking places,ADJUSTED,17491.0,1992-12-01,1992,12


Describint the master raw data

In [90]:
master_raw_data.describe()

,index,Value,Year,Month
count,36050.000000,35447.000000,36050.000000,36050.000000
mean,27.650485,50020.494767,2006.085714,6.471429
std,17.761177,93994.323779,8.419046,3.463001
min,0.000000,11.000000,1992.000000,1.000000
25%,13.000000,3451.000000,1999.000000,3.000000
50%,26.000000,12444.000000,2006.000000,6.000000
75%,40.000000,41882.000000,2013.000000,9.000000
max,65.000000,611429.000000,2021.000000,12.000000


[Back to top](#Index)

## 4. Finalizing the data and preparing the Installation Scripts

Creating the Insert scripts from the data frame and writing to the MRTS_SALES_DATA.SQL file

In [91]:
insertScriptLines = []
#first line to be inserted in the output file to connect to the database which is used in the Schema file
dbConnectInit = 'USE MRTSSALES;'
insertScriptLines.append(dbConnectInit + '\n')

#Defaulting the NAICS Code to Total for the Total rows
master_raw_data.NAICS_Code.fillna('Total',inplace = True)
master_raw_data.Value.fillna(0,inplace = True)

#Looping through the master_raw_data to create the Insert script for each row
for index,row in master_raw_data.iterrows():
    temp = ""
    temp = "INSERT INTO MRTSSALES_DATA (NAICS_CODE, KIND_OF_BUSINESS, ADJUSTMENT_TYPE, VALUE, DATE) VALUES('" + row['NAICS_Code'] + "','" + row['Kind_of_Business'].replace("'","''") + "','" + row['Adjustment_Type'] + "'," + str(row['Value']) + ",'" + row['Date'].strftime("%Y-%m-%d") + "');"
    insertScriptLines.append(temp + '\n')

#Opening the output file to write
file = open('MRTS_SALES_DATA.SQL', 'r+')

#delete the existing content in the file
file.truncate(0)

# Writing multiple Lines at a time
file.writelines(insertScriptLines)

# Closing file
file.close()

[Back to top](#Index)

## 5. Conclusion

the Installation file for loadin the data is ready to use.

In [92]:
print('SQL File Written Completed')

SQL File Written Completed


[Back to top](#Index)

## 6. References

Adding the References which have been used in this notebook

- Sayon, Shubham “File Writing Methods' https://blog.finxter.com/correct-way-to-write-line-to-file-in-python/

- “Reading the Excel using Pandas” https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
